In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col

In [3]:
# Initialize a Spark session
spark = SparkSession.builder.appName("DataTransformation").getOrCreate()

23/09/05 13:24:37 WARN Utils: Your hostname, ubuntu-Lenovo-Legion-5-15ARH05 resolves to a loopback address: 127.0.1.1; using 172.16.5.112 instead (on interface wlp4s0)
23/09/05 13:24:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/05 13:24:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Load the CSV data into DataFrames
listing_df = spark.read.csv('raw_data/listings.csv', header=True, inferSchema=True)
reviews_df = spark.read.csv('raw_data/reviews.csv', header=True, inferSchema=True)
calendar_df = spark.read.csv('raw_data/calendar.csv', header=True, inferSchema=True)


In [6]:
# Transform the data

# LISTINGS DATA

# Drop the 'summary' column
listing_df = listing_df.drop('summary')
# Convert 'host_is_superhost' to boolean
listing_df = listing_df.withColumn('host_is_superhost', when(col('host_is_superhost') == 't', True).otherwise(False))

# Drop 'country' and 'market' columns
listing_df = listing_df.drop('country', 'market')

# Drop rows with null values in the 'space' column
listing_df = listing_df.na.drop(subset=['space'])

# Fill null values in 'host_about' with "No description"
listing_df = listing_df.na.fill({'host_about': 'No description'})

In [7]:
# CALENDAR DATA

# Fill null values in 'price' column with "Booked"
calendar_df = calendar_df.na.fill({'price': 'Booked'})

# Convert 'available' to boolean
calendar_df = calendar_df.withColumn('available', when(col('available') == 't', True).otherwise(False))

In [9]:
# REVIEW DATA

# No transformation needed

# SAVE DATA

listing_df.write.csv('cleaned_data/clean_listing_csv', header=True, mode="overwrite")
calendar_df.write.csv('cleaned_data/clean0_calendar_csv', header=True, mode="overwrite")
reviews_df.write.csv('cleaned_data/clean_reviews_csv', header=True, mode="overwrite")

In [10]:
# Use the read method to read the CSV file into a DataFrame
# read_df = spark.read.csv('cleaned_data/clean_listing_csv/', header=True, inferSchema=True)
# read_df.show(10)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------------+------------+---------+--------+-------+-----------------+
|                  id|         listing_url|                name|               space|         description|           host_name|       host_location|          host_about|  host_response_time|  host_response_rate|host_acceptance_rate|host_is_superhost|          host_since|              street|                city|               state|             zipcode|       property_type|      room_type|accommodates|bathrooms|bedrooms|  price|number_of_reviews|
+--------------------+--------------------+--------------------+--------------------+-------------